# Kimore

## import dataset

In [ ]:
import os
import csv
import openpyxl
import pandas as pd
import numpy as np

path = "Kimore"
kinect_joints = ["spinebase", "spinemid", "neck", "head", 
                 "shoulderleft", "elbowleft", "wristleft", 
                 "handleft", "shoulderright", "elbowright", 
                 "wristright", "handright", "hipleft", "kneeleft", 
                 "ankleleft", "footleft", "hipright", "kneeright", 
                 "ankleright", "footright", "spineshoulder", "handtipleft", 
                 "thumbleft", "handtipright", "thumbright"]

enable_kinect_joints = False
enable_slice_list = False

data = []
for (root, dirs, files) in os.walk(path):

    # if current directory contains "Raw", extract data
    if "Raw" in dirs:
        
        new_dict = {}
        
        # get exercise number
        new_dict["Exercise"] = int(root[-1])

        # extract raw data
        raw_files = os.listdir(os.path.join(root, "Raw"))
        for file in raw_files:

            file_path = os.path.join(os.path.join(root, "Raw"),file)
            csv_file = open(file_path, newline='')
            csv_reader = csv.reader(csv_file)
            
            if file.startswith("JointOrientation"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-o"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-o"].append(row[(4*i):(4*i+4)])
                else:
                    new_dict["JointOrientation"] = []
                    for row in csv_reader:
                        if len(new_dict["JointOrientation"]) >= 182 and enable_slice_list:
                            break
                        elif len(row) > 0:
                            if '' in row:
                                row.remove('')
                            new_dict["JointOrientation"].append(np.array([float(i) for i in row]))
                    np.array(new_dict["JointOrientation"])
                            

            elif file.startswith("JointPosition"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-p"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-p"].append(row[(4*i):(4*i+3)])
                else:
                    new_dict["JointPosition"] = []
                    for row in csv_reader:
                        if len(new_dict["JointPosition"]) >= 182 and enable_slice_list:
                            break
                        elif len(row) > 0:
                            if '' in row:
                                row.remove('')
                            new_dict["JointPosition"].append(np.array([float(i) for i in row]))
                    np.array(new_dict["JointPosition"])

            elif file.startswith("TimeStamp"):

                new_dict["Timestamps"] = []
                for row in csv_reader:
                    if len(new_dict["Timestamps"]) >= 182 and enable_slice_list:
                        break
                    elif len(row) > 0:
                        if '' in row:
                                row.remove('')
                        new_dict["Timestamps"].append(row)
                            
                        

        # extract data labels
        label_files = os.listdir(os.path.join(root, "Label"))
        for file in label_files:

            file_path = os.path.join(os.path.join(root, "Label"),file)
            book = openpyxl.load_workbook(file_path)
            sheet = book.active

            if file.startswith("SuppInfo"):
                for i in range(1, sheet.max_column):
                    t = sheet.cell(1, i).value
                    v = sheet.cell(2, i).value
                    new_dict[t] = v
            
            elif file.startswith("ClinicalAssessment"):
                new_dict["cTS"] = sheet.cell(2, new_dict["Exercise"]+1).value
                new_dict["cPO"] = sheet.cell(2, new_dict["Exercise"]+6).value
                new_dict["cCF"] = sheet.cell(2, new_dict["Exercise"]+11).value
        
        # append exercise to data
        data.append(new_dict)

## functions

In [2]:
def findOutlierLen(JP, JO):
    """
    Parameters:
        JP: JointPosition.
        JO: JointOrientation.
    """
    signal = 1
    outlier = []
    for i in range(0, len(JP)):
        for j in range(0, len(JP[i])):
            if len(JP[i][j]) != 100:
                signal = 0
                outlier.append(i)
                break
        
    if signal:
        print("No outlier lenght.")
        
    return outlier
      
    
def isSameShape(JP, JO):
    """
    Parameters:
        JP: JointPosition.
        JO: JointOrientation.
    """
    signal = 1
    for i in range(0, len(JP)):
        for j in range(0, len(JP[i])):
            if np.shape(JP) != np.shape(JO):
                signal = 0
                print("Position:\t", i, j, "len: ",len(JP[i][j]))
                print("Orientation:\t", i, j, "len: ",len(JO[i][j]))

    if signal:
        print("All equally shaped.")

        
def minMaxLen(TM):
    """
    Parameters:
        TM: Timestamps.
    """
    print("min len: ", TM.apply(len).sort_values().iat[0], "idx: ", TM.apply(len).sort_values().index[0],
      "\nmax len: ", TM.apply(len).sort_values(ascending=False).iat[0], "idx: ", TM.apply(len).sort_values(ascending=False).index[0])
    
    
def groupExerciseCount(df):
    """
    Parameters:
        df: Dataframe
    """
    for g in df.Group.unique():
        for e in df.Exercise.unique():
            print("Group:", g, "\tExercise:", e, "\tCount:", df[(df["Group"]==g) & (df["Exercise"]==e)].shape[0])

In [3]:
df = pd.DataFrame(data)

In [4]:
df

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
0,1,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2.0, -0.04849...","[[20622112895], [20622442942], [20622783803], ...",48.333333,14.666667,33.666667,E_ID1,E,50
1,2,"[[-0.0610258, 0.00258101, 0.995022, -0.0787486...","[[-0.0578279, -0.407365, 2.60017, 2.0, -0.0592...","[[21184779662], [21185108840], [21185438830], ...",45.000000,13.666667,31.333333,E_ID1,E,50
2,3,"[[-0.0298744, -0.00814489, 0.995711, -0.087179...","[[-0.0652774, -0.418544, 2.56966, 2.0, -0.0718...","[[21902433786], [21902774551], [21903104529], ...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
3,4,"[[-0.0369213, -0.00490366, 0.995614, -0.085821...","[[-0.0674865, -0.397115, 2.58841, 2.0, -0.0723...","[[22621099365], [22621428603], [22621769446], ...",50.000000,15.000000,35.000000,E_ID1,CG_E,50
4,5,"[[-0.0517414, -0.000435555, 0.99603, -0.072440...","[[-0.0630149, -0.394823, 2.59342, 2.0, -0.0655...","[[23248424127], [23248764907], [23249094898], ...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
...,...,...,...,...,...,...,...,...,...,...
385,1,"[[0.039838, -0.0277761, 0.995785, -0.0777996, ...","[[-0.20825, -0.500367, 2.32432, 2.0, -0.223546...","[[711302136484], [711302477267], [711302806451...",26.333333,6.333333,20.000000,S_ID9,S,67
386,2,"[[0.0600943, -0.0332151, 0.995777, -0.0609322,...","[[-0.209425, -0.48988, 2.31622, 2.0, -0.227487...","[[711894802395], [711895132382], [711895473180...",17.666667,4.333333,13.333333,S_ID9,S,67
387,3,"[[0.0795781, -0.0310581, 0.994218, -0.0650627,...","[[-0.202571, -0.492478, 2.31694, 2.0, -0.21846...","[[712329470249], [712329799436], [712330129378...",26.333333,6.333333,20.000000,S_ID9,S,67
388,4,"[[-0.00995722, -0.0160432, 0.997548, -0.067393...","[[-0.199238, -0.471849, 2.29448, 2.0, -0.20909...","[[712712796766], [712713126759], [712713467555...",23.000000,7.000000,16.000000,S_ID9,S,67


In [5]:
df.at[2, 'Group'] = "E"
df.at[3, 'Group'] = "E"
df.at[4, 'Group'] = "E"

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Exercise,390.0,3.000000,1.416030,1.000000,2.000000,3.000000,4.0,5.0
cTS,383.0,37.457573,10.204780,10.000000,30.140452,40.000000,46.0,50.0
cPO,383.0,11.040233,3.952683,0.005147,8.540081,12.000000,14.5,15.0
cCF,383.0,26.413858,7.043139,7.000000,22.276553,27.360633,32.0,35.0
Age,390.0,45.641026,19.354609,17.000000,26.000000,45.500000,63.0,81.0


In [7]:
df.isnull().sum()

Exercise            0
JointOrientation    5
JointPosition       5
Timestamps          3
cTS                 7
cPO                 7
cCF                 7
Subject ID          0
Group               0
Age                 0
dtype: int64

In [8]:
dropped_rows = df[df.isna().any(axis=1)]
dropped_rows

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
40,1,"[[-0.0481282, 8.87374e-07, 0.985768, 0.161073,...","[[0.175871, -1.10702, 2.84095, 2.0, 0.180718, ...","[[30437599246420], [30437599576380], [30437599...",NaN,NaN,NaN,E_ID17,E,45
41,2,"[[0.0199295, -0.0368523, 0.996205, 0.0762919, ...","[[0.155424, -1.09849, 3.29907, 2.0, 0.13016, -...","[[30438448240198], [30438448570198], [30438448...",NaN,NaN,NaN,E_ID17,E,45
42,3,"[[-0.228232, 0.030343, 0.964276, 0.131002, -0....","[[0.173645, -1.03028, 3.27985, 2.0, 0.209115, ...","[[30439025566426], [30439025896688], [30439026...",NaN,NaN,NaN,E_ID17,E,45
43,4,"[[-0.101832, 0.0088296, 0.98639, 0.128791, -0....","[[0.277114, -1.03271, 3.01552, 2.0, 0.290556, ...","[[30442081214560], [30442081543751], [30442081...",NaN,NaN,NaN,E_ID17,E,45
44,5,"[[-0.0201885, -0.0603588, 0.992337, -0.105904,...","[[0.300576, -1.3918, 3.14916, 2.0, 0.263258, -...","[[30442714879944], [30442715209965], [30442715...",NaN,NaN,NaN,E_ID17,E,45
76,2,NaN,NaN,NaN,NaN,NaN,NaN,E_ID8,E,22
97,3,NaN,"[[-0.196504, -0.761454, 2.62517, 2.0, -0.19706...","[[10369220941503], [10369221270654], [10369221...",40.00000,15.000000,25.000000,NE_ID11,NE,75
99,5,[],NaN,"[[10370694259997], [10370694589964], [10370694...",49.00000,14.000000,35.000000,NE_ID11,NE,75
166,2,"[[0.0367658, 0.00355401, 0.998187, -0.0475269,...",NaN,"[[184099250059], [184099580040], [184099910803...",44.00000,13.000000,31.000000,NE_ID24,NE,25
258,4,NaN,NaN,NaN,NaN,NaN,NaN,B_ID8,B,36


In [9]:
del dropped_rows

In [10]:
df = df.dropna().reset_index(drop=True)

In [11]:
outlier = findOutlierLen(df.JointPosition, df.JointOrientation)
if len(outlier) > 0:
    df = df.drop(outlier).reset_index(drop=True)
    outlier = findOutlierLen(df.JointPosition, df.JointOrientation)
del outlier

No outlier lenght.


In [12]:
isSameShape(df.JointPosition, df.JointOrientation)

All equally shaped.


In [13]:
minMaxLen(df.Timestamps)

min len:  182 idx:  26 
max len:  2487 idx:  327


In [14]:
groupExerciseCount(df)

Group: E 	Exercise: 1 	Count: 15
Group: E 	Exercise: 2 	Count: 15
Group: E 	Exercise: 3 	Count: 16
Group: E 	Exercise: 4 	Count: 16
Group: E 	Exercise: 5 	Count: 16
Group: NE 	Exercise: 1 	Count: 27
Group: NE 	Exercise: 2 	Count: 26
Group: NE 	Exercise: 3 	Count: 26
Group: NE 	Exercise: 4 	Count: 27
Group: NE 	Exercise: 5 	Count: 26
Group: B 	Exercise: 1 	Count: 8
Group: B 	Exercise: 2 	Count: 8
Group: B 	Exercise: 3 	Count: 8
Group: B 	Exercise: 4 	Count: 7
Group: B 	Exercise: 5 	Count: 8
Group: P 	Exercise: 1 	Count: 15
Group: P 	Exercise: 2 	Count: 16
Group: P 	Exercise: 3 	Count: 16
Group: P 	Exercise: 4 	Count: 16
Group: P 	Exercise: 5 	Count: 15
Group: S 	Exercise: 1 	Count: 10
Group: S 	Exercise: 2 	Count: 10
Group: S 	Exercise: 3 	Count: 9
Group: S 	Exercise: 4 	Count: 10
Group: S 	Exercise: 5 	Count: 9


## windowing

In [15]:
from numpy.lib.stride_tricks import sliding_window_view

def windowing(joints, shape = 100):
    v = sliding_window_view(np.array(joints[0:len(joints) - (len(joints) % 100)]).flatten(), shape)
    return v[::v.shape[0]//500,:]

In [16]:
df.JointPosition = df.JointPosition.apply(lambda x: windowing(x)[:500, :]).to_numpy()
df.JointOrientation = df.JointOrientation.apply(lambda x: windowing(x)[:500, :]).to_numpy()

In [17]:
list_of_group = []
for g in df.Group.unique():
    for e in df.Exercise.unique():
        subset = df.loc[(df["Group"] == g) & (df["Exercise"] == e), :].sample(7)
        list_of_group.append(subset)
        
downsampled_df = pd.concat(list_of_group, ignore_index=True)
del list_of_group

In [18]:
downsampled_df = downsampled_df.drop(columns=["Timestamps", "cTS", "cPO", "cCF",  "Age "])
e = downsampled_df.pop("Exercise")
g = downsampled_df.pop("Group")
downsampled_df["Exercise"] = e
downsampled_df["Group"] = g
del e
del g
downsampled_df

,JointOrientation,JointPosition,Subject ID,Exercise,Group
0,"[[-0.0020808, -0.00690942, 0.997573, -0.069259...","[[-0.120613, -0.255417, 2.52227, 2.0, -0.12552...",E_ID11,1,E
1,"[[-0.00766952, -0.0174588, 0.995782, -0.089742...","[[-0.167507, -0.427232, 2.35279, 2.0, -0.17801...",E_ID10,1,E
2,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2.0, -0.04849...",E_ID1,1,E
3,"[[-0.0431788, -0.0317104, 0.998489, 0.0122701,...","[[-0.0750939, -0.448098, 2.71395, 2.0, -0.0948...",E_ID15,1,E
4,"[[-0.0179779, 0.00536922, 0.996446, -0.0821169...","[[-0.0811335, -0.28634, 2.31876, 2.0, -0.07840...",E_ID6,1,E
...,...,...,...,...,...
170,"[[0.0994289, -0.0135622, 0.993961, -0.0443926,...","[[-0.551777, -0.653763, 3.37547, 2.0, -0.55710...",S_ID1,5,S
171,"[[-0.0252481, -0.00317248, 0.995375, -0.092637...","[[-0.360684, -0.255174, 2.52094, 2.0, -0.36413...",S_ID7,5,S
172,"[[0.0636762, -0.00561611, 0.992197, -0.107044,...","[[-0.413131, -1.08097, 3.33802, 2.0, -0.412387...",S_ID2,5,S
173,"[[0.131028, 0.0116614, 0.987266, 0.0894549, 0....","[[-0.151124, -0.746975, 3.43042, 2.0, -0.15124...",S_ID4,5,S


In [19]:
groupExerciseCount(downsampled_df)

Group: E 	Exercise: 1 	Count: 7
Group: E 	Exercise: 2 	Count: 7
Group: E 	Exercise: 3 	Count: 7
Group: E 	Exercise: 4 	Count: 7
Group: E 	Exercise: 5 	Count: 7
Group: NE 	Exercise: 1 	Count: 7
Group: NE 	Exercise: 2 	Count: 7
Group: NE 	Exercise: 3 	Count: 7
Group: NE 	Exercise: 4 	Count: 7
Group: NE 	Exercise: 5 	Count: 7
Group: B 	Exercise: 1 	Count: 7
Group: B 	Exercise: 2 	Count: 7
Group: B 	Exercise: 3 	Count: 7
Group: B 	Exercise: 4 	Count: 7
Group: B 	Exercise: 5 	Count: 7
Group: P 	Exercise: 1 	Count: 7
Group: P 	Exercise: 2 	Count: 7
Group: P 	Exercise: 3 	Count: 7
Group: P 	Exercise: 4 	Count: 7
Group: P 	Exercise: 5 	Count: 7
Group: S 	Exercise: 1 	Count: 7
Group: S 	Exercise: 2 	Count: 7
Group: S 	Exercise: 3 	Count: 7
Group: S 	Exercise: 4 	Count: 7
Group: S 	Exercise: 5 	Count: 7


In [20]:
minMaxLen(downsampled_df.JointOrientation)

min len:  500 idx:  0 
max len:  500 idx:  0


In [21]:
def slice_joints(df):
    temp = pd.DataFrame()
    temp["JointOrientation"] = np.array_split(df.JointOrientation, 10)
    temp["JointPosition"] = np.array_split(df.JointPosition, 10)
    temp["Subject ID"] = np.repeat(df["Subject ID"], 10)
    temp["Exercise"] = np.repeat(df.Exercise, 10)
    temp["Group"] = np.repeat(df.Group, 10)
    
    return temp

In [22]:
list_of_rows = []
for _, row in downsampled_df.iterrows():
    list_of_rows.append(slice_joints(row))
new_df = pd.concat(list_of_rows, ignore_index=True)
del list_of_rows
del downsampled_df
new_df

,JointOrientation,JointPosition,Subject ID,Exercise,Group
0,"[[-0.0020808, -0.00690942, 0.997573, -0.069259...","[[-0.120613, -0.255417, 2.52227, 2.0, -0.12552...",E_ID11,1,E
1,"[[0.7042, -0.203354, 0.0895451, 0.711398, -0.1...","[[2.43267, 2.0, -0.246503, -0.619447, 2.57676,...",E_ID11,1,E
2,"[[-0.0124066, 0.00102059, 0.996435, -0.0834452...","[[-0.125476, -0.216276, 2.51728, 2.0, -0.12548...",E_ID11,1,E
3,"[[0.709775, -0.215936, 0.0787758, 0.716372, -0...","[[2.45385, 2.0, -0.247389, -0.616206, 2.58684,...",E_ID11,1,E
4,"[[-0.0124606, 0.00440577, 0.997262, -0.0727568...","[[-0.126244, -0.228651, 2.5382, 2.0, -0.123814...",E_ID11,1,E
...,...,...,...,...,...
1745,"[[0.754799, -0.368708, 0.209744, -0.531221, -0...","[[2.30017, 2.0, -0.522438, -0.870507, 2.13855,...",S_ID8,5,S
1746,"[[-0.0241948, 0.0228407, 0.970382, -0.239275, ...","[[-0.345708, -0.441526, 2.1746, 2.0, -0.335812...",S_ID8,5,S
1747,"[[0.741577, -0.315374, 0.092705, -0.576227, 0....","[[2.18036, 2.0, -0.507034, -0.780764, 2.17768,...",S_ID8,5,S
1748,"[[0.00647624, 0.0189398, 0.971545, -0.236008, ...","[[-0.348254, -0.434848, 2.20254, 2.0, -0.33618...",S_ID8,5,S


In [23]:
outlier = findOutlierLen(new_df.JointPosition, new_df.JointOrientation)
del outlier

No outlier lenght.


In [24]:
isSameShape(new_df.JointPosition, new_df.JointOrientation)

All equally shaped.


In [25]:
minMaxLen(new_df.JointPosition)

min len:  50 idx:  0 
max len:  50 idx:  0


In [26]:
groupExerciseCount(new_df)

Group: E 	Exercise: 1 	Count: 70
Group: E 	Exercise: 2 	Count: 70
Group: E 	Exercise: 3 	Count: 70
Group: E 	Exercise: 4 	Count: 70
Group: E 	Exercise: 5 	Count: 70
Group: NE 	Exercise: 1 	Count: 70
Group: NE 	Exercise: 2 	Count: 70
Group: NE 	Exercise: 3 	Count: 70
Group: NE 	Exercise: 4 	Count: 70
Group: NE 	Exercise: 5 	Count: 70
Group: B 	Exercise: 1 	Count: 70
Group: B 	Exercise: 2 	Count: 70
Group: B 	Exercise: 3 	Count: 70
Group: B 	Exercise: 4 	Count: 70
Group: B 	Exercise: 5 	Count: 70
Group: P 	Exercise: 1 	Count: 70
Group: P 	Exercise: 2 	Count: 70
Group: P 	Exercise: 3 	Count: 70
Group: P 	Exercise: 4 	Count: 70
Group: P 	Exercise: 5 	Count: 70
Group: S 	Exercise: 1 	Count: 70
Group: S 	Exercise: 2 	Count: 70
Group: S 	Exercise: 3 	Count: 70
Group: S 	Exercise: 4 	Count: 70
Group: S 	Exercise: 5 	Count: 70


## train-test-validation

In [27]:
x1 = np.array(list(zip(new_df.JointOrientation, new_df.JointPosition)))

In [28]:
x1.shape

(1750, 2, 50, 100)

In [29]:
x1 = np.transpose(x1, (0, 2, 3, 1))
x1.shape

(1750, 50, 100, 2)

In [30]:
x2 = np.concatenate([np.stack(new_df.JointOrientation),np.stack(new_df.JointPosition)], axis=-1)
x2.shape

(1750, 50, 200)

In [31]:
y = new_df.Exercise.to_numpy()
y -=1
y.shape

(1750,)

In [32]:
def get_dataset_partitions(x, y, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1
    
    # Only allows for equal validation and test splits
    # assert val_split == test_split 
    
    assert len(x) == len(y)
    # Specify seed to always have the same split distribution between runs
    p = np.random.permutation(len(y))
    indices_or_sections = [int(train_split * len(y)), int((1 - val_split - test_split) * len(y))]
    
    train_x, val_x, test_x = np.split(x[p], indices_or_sections)
    train_y, val_y, test_y = np.split(y[p], indices_or_sections)
    
    return train_x, val_x, test_x, train_y, val_y, test_y

train_x, val_x, test_x, train_y, val_y, test_y = get_dataset_partitions(x1, y, val_split = 0, test_split = 0.2)

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt

C:\Users\Dilemre\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [34]:
input_shape = (50, 100, 2)

## acc 0.68
|layer|node|activation|
|-|-|-|
|conv|16|relu|
|lstm|16|tanh|
|dense|5||

In [35]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Reshape((200, 100)))
model.add(LSTM(16, activation="tanh"))
model.add(Dense(5))

In [36]:
model.build(input_shape=input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 100, 16)       144       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 50, 16)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 reshape (Reshape)           (None, 200, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 16)                7488      
                                                                 
 dense (Dense)               (None, 5)                 85        
                                                        

In [37]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [38]:
history = model.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 5s 96ms/step - loss: 1.6230 - accuracy: 0.1973 - val_loss: 1.6123 - val_accuracy: 0.1893
Epoch 2/100
35/35 [==============================] - 3s 77ms/step - loss: 1.6068 - accuracy: 0.2259 - val_loss: 1.6090 - val_accuracy: 0.1929
Epoch 3/100
35/35 [==============================] - 2s 72ms/step - loss: 1.6039 - accuracy: 0.2429 - val_loss: 1.6100 - val_accuracy: 0.2250
Epoch 4/100
35/35 [==============================] - 2s 71ms/step - loss: 1.5879 - accuracy: 0.3000 - val_loss: 1.5973 - val_accuracy: 0.2250
Epoch 5/100
35/35 [==============================] - 3s 72ms/step - loss: 1.5922 - accuracy: 0.2625 - val_loss: 1.5942 - val_accuracy: 0.2107
Epoch 6/100
35/35 [==============================] - 3s 73ms/step - loss: 1.5739 - accuracy: 0.3000 - val_loss: 1.5823 - val_accuracy: 0.2500
Epoch 7/100
35/35 [==============================] - 3s 72ms/step - loss: 1.5532 - accuracy: 0.3527 - val_loss: 1.5615 - val_accuracy: 0.2679
Epoch 

35/35 [==============================] - 3s 73ms/step - loss: 0.5075 - accuracy: 0.8125 - val_loss: 0.8473 - val_accuracy: 0.6679
Epoch 59/100
35/35 [==============================] - 3s 73ms/step - loss: 0.4693 - accuracy: 0.8411 - val_loss: 0.7901 - val_accuracy: 0.7036
Epoch 60/100
35/35 [==============================] - 3s 73ms/step - loss: 0.5569 - accuracy: 0.8080 - val_loss: 0.8047 - val_accuracy: 0.6929
Epoch 61/100
35/35 [==============================] - 3s 73ms/step - loss: 0.4594 - accuracy: 0.8357 - val_loss: 0.8753 - val_accuracy: 0.6714
Epoch 62/100
35/35 [==============================] - 3s 76ms/step - loss: 0.4460 - accuracy: 0.8464 - val_loss: 0.8103 - val_accuracy: 0.7357
Epoch 63/100
35/35 [==============================] - 3s 73ms/step - loss: 0.4131 - accuracy: 0.8607 - val_loss: 0.7899 - val_accuracy: 0.6929
Epoch 64/100
35/35 [==============================] - 3s 81ms/step - loss: 0.3744 - accuracy: 0.8821 - val_loss: 0.7926 - val_accuracy: 0.7000
Epoch 65/100

In [39]:
 model.evaluate(test_x, test_y)

11/11 [==============================] - 0s 20ms/step - loss: 0.9304 - accuracy: 0.7257


[0.9303936958312988, 0.7257142663002014]

## acc 0.51
|layer|node|activation|
|-|-|-|
|conv|32|relu|
|lstm|16|softmax|
|dense|5||

In [40]:
model2 = Sequential()
model2.add(Conv2D(32, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Reshape((400, 100)))
model2.add(LSTM(32, activation="softmax"))
model2.add(Dense(5))
model2.build(input_shape=input_shape)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 50, 100, 32)       288       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 40000)             0         
                                                                 
 reshape_1 (Reshape)         (None, 400, 100)          0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                17024     
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                      

In [41]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
history2 = model2.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 9s 218ms/step - loss: 1.6101 - accuracy: 0.1973 - val_loss: 1.6098 - val_accuracy: 0.1929
Epoch 2/100
35/35 [==============================] - 7s 210ms/step - loss: 1.6094 - accuracy: 0.2116 - val_loss: 1.6095 - val_accuracy: 0.2071
Epoch 3/100
35/35 [==============================] - 7s 195ms/step - loss: 1.6089 - accuracy: 0.2116 - val_loss: 1.6096 - val_accuracy: 0.2357
Epoch 4/100
35/35 [==============================] - 7s 190ms/step - loss: 1.6081 - accuracy: 0.2500 - val_loss: 1.6094 - val_accuracy: 0.2214
Epoch 5/100
35/35 [==============================] - 7s 190ms/step - loss: 1.6043 - accuracy: 0.2384 - val_loss: 1.6102 - val_accuracy: 0.1786
Epoch 6/100
35/35 [==============================] - 7s 190ms/step - loss: 1.5921 - accuracy: 0.2446 - val_loss: 1.6067 - val_accuracy: 0.3107
Epoch 7/100
35/35 [==============================] - 7s 190ms/step - loss: 1.5789 - accuracy: 0.3348 - val_loss: 1.5967 - val_accuracy: 0.3000

Epoch 58/100
35/35 [==============================] - 7s 202ms/step - loss: 1.3080 - accuracy: 0.3938 - val_loss: 1.3757 - val_accuracy: 0.3393
Epoch 59/100
35/35 [==============================] - 7s 192ms/step - loss: 1.2955 - accuracy: 0.4089 - val_loss: 1.3677 - val_accuracy: 0.3500
Epoch 60/100
35/35 [==============================] - 7s 192ms/step - loss: 1.3000 - accuracy: 0.4143 - val_loss: 1.3551 - val_accuracy: 0.3393
Epoch 61/100
35/35 [==============================] - 7s 192ms/step - loss: 1.2807 - accuracy: 0.4125 - val_loss: 1.3396 - val_accuracy: 0.3893
Epoch 62/100
35/35 [==============================] - 7s 194ms/step - loss: 1.2742 - accuracy: 0.4062 - val_loss: 1.3260 - val_accuracy: 0.3893
Epoch 63/100
35/35 [==============================] - 7s 191ms/step - loss: 1.2592 - accuracy: 0.4357 - val_loss: 1.3060 - val_accuracy: 0.4286
Epoch 64/100
35/35 [==============================] - 7s 194ms/step - loss: 1.2396 - accuracy: 0.4536 - val_loss: 1.2943 - val_accuracy:

In [43]:
 model2.evaluate(test_x, test_y)

11/11 [==============================] - 1s 59ms/step - loss: 1.0896 - accuracy: 0.5400


[1.0896130800247192, 0.5400000214576721]

## acc 0.68
|layer|node|activation|
|-|-|-|
|conv|32|relu|
|lstm|32|tanh|
|dense|5|softmax|

In [44]:
model3 = Sequential()
model3.add(Conv2D(32, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Reshape((400, 100)))
model3.add(LSTM(32, activation="tanh"))
model3.add(Dense(5, activation="softmax"))
model3.build(input_shape=input_shape)
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 50, 100, 32)       288       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 40000)             0         
                                                                 
 reshape_2 (Reshape)         (None, 400, 100)          0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                17024     
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                      

In [45]:
model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [46]:
history3 = model3.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 9s 197ms/step - loss: 1.6158 - accuracy: 0.1866 - val_loss: 1.6088 - val_accuracy: 0.1964
Epoch 2/100
35/35 [==============================] - 6s 175ms/step - loss: 1.6041 - accuracy: 0.2170 - val_loss: 1.6080 - val_accuracy: 0.1893
Epoch 3/100
35/35 [==============================] - 6s 185ms/step - loss: 1.5801 - accuracy: 0.2705 - val_loss: 1.5869 - val_accuracy: 0.2714
Epoch 4/100
35/35 [==============================] - 6s 179ms/step - loss: 1.5417 - accuracy: 0.3187 - val_loss: 1.5432 - val_accuracy: 0.3250
Epoch 5/100
35/35 [==============================] - 6s 178ms/step - loss: 1.4491 - accuracy: 0.3634 - val_loss: 1.4765 - val_accuracy: 0.3143
Epoch 6/100
35/35 [==============================] - 6s 171ms/step - loss: 1.3656 - accuracy: 0.3777 - val_loss: 1.4466 - val_accuracy: 0.3714
Epoch 7/100
35/35 [==============================] - 6s 171ms/step - loss: 1.3566 - accuracy: 0.4062 - val_loss: 1.4925 - val_accuracy: 0.2929

Epoch 58/100
35/35 [==============================] - 6s 176ms/step - loss: 0.5125 - accuracy: 0.8018 - val_loss: 1.1464 - val_accuracy: 0.6179
Epoch 59/100
35/35 [==============================] - 6s 173ms/step - loss: 0.5206 - accuracy: 0.7937 - val_loss: 1.2039 - val_accuracy: 0.5964
Epoch 60/100
35/35 [==============================] - 6s 171ms/step - loss: 0.4842 - accuracy: 0.8241 - val_loss: 1.1822 - val_accuracy: 0.6214
Epoch 61/100
35/35 [==============================] - 6s 171ms/step - loss: 0.4895 - accuracy: 0.8098 - val_loss: 1.0955 - val_accuracy: 0.6214
Epoch 62/100
35/35 [==============================] - 6s 171ms/step - loss: 0.4390 - accuracy: 0.8429 - val_loss: 1.1592 - val_accuracy: 0.6143
Epoch 63/100
35/35 [==============================] - 6s 171ms/step - loss: 0.4941 - accuracy: 0.8116 - val_loss: 1.1135 - val_accuracy: 0.5786
Epoch 64/100
35/35 [==============================] - 6s 171ms/step - loss: 0.4490 - accuracy: 0.8313 - val_loss: 1.1836 - val_accuracy:

In [47]:
 model3.evaluate(test_x, test_y)

11/11 [==============================] - 1s 46ms/step - loss: 1.1436 - accuracy: 0.7114


[1.1436259746551514, 0.7114285826683044]

## acc 0.88
|layer|node|activation|
|-|-|-|
|conv|32|relu|
|conv|64|relu|
|lstm|32|tanh|
|dense|5|softmax|

In [48]:
model4 = Sequential()
model4.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Reshape((-1, 64)))
model4.add(LSTM(32, activation="tanh"))
model4.add(Dense(5, activation="softmax"))
model4.build(input_shape=input_shape)
model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 50, 100, 32)       608       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 25, 50, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 25, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 19200)             0         
                                                                 
 reshape_3 (Reshape)         (None, 300, 64)          

In [49]:
model4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [50]:
history4 = model4.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 8s 192ms/step - loss: 1.6168 - accuracy: 0.2143 - val_loss: 1.6010 - val_accuracy: 0.2179
Epoch 2/100
35/35 [==============================] - 6s 179ms/step - loss: 1.5545 - accuracy: 0.2929 - val_loss: 1.5237 - val_accuracy: 0.2929
Epoch 3/100
35/35 [==============================] - 7s 199ms/step - loss: 1.4314 - accuracy: 0.3589 - val_loss: 1.5104 - val_accuracy: 0.3000
Epoch 4/100
35/35 [==============================] - 8s 217ms/step - loss: 1.3402 - accuracy: 0.4000 - val_loss: 1.4333 - val_accuracy: 0.3464
Epoch 5/100
35/35 [==============================] - 7s 190ms/step - loss: 1.2825 - accuracy: 0.4437 - val_loss: 1.3188 - val_accuracy: 0.4214
Epoch 6/100
35/35 [==============================] - 7s 196ms/step - loss: 1.2364 - accuracy: 0.4527 - val_loss: 1.2964 - val_accuracy: 0.4179
Epoch 7/100
35/35 [==============================] - 6s 180ms/step - loss: 1.2062 - accuracy: 0.4804 - val_loss: 1.2638 - val_accuracy: 0.4536

Epoch 58/100
35/35 [==============================] - 6s 177ms/step - loss: 0.4157 - accuracy: 0.8652 - val_loss: 0.5288 - val_accuracy: 0.8143
Epoch 59/100
35/35 [==============================] - 7s 201ms/step - loss: 0.4241 - accuracy: 0.8625 - val_loss: 0.5718 - val_accuracy: 0.8071
Epoch 60/100
35/35 [==============================] - 7s 187ms/step - loss: 0.4063 - accuracy: 0.8643 - val_loss: 0.6080 - val_accuracy: 0.8250
Epoch 61/100
35/35 [==============================] - 6s 173ms/step - loss: 0.8095 - accuracy: 0.7598 - val_loss: 1.5879 - val_accuracy: 0.5321
Epoch 62/100
35/35 [==============================] - 6s 173ms/step - loss: 0.7704 - accuracy: 0.7536 - val_loss: 0.7276 - val_accuracy: 0.7500
Epoch 63/100
35/35 [==============================] - 6s 172ms/step - loss: 0.4884 - accuracy: 0.8429 - val_loss: 0.5826 - val_accuracy: 0.7964
Epoch 64/100
35/35 [==============================] - 6s 176ms/step - loss: 0.4287 - accuracy: 0.8679 - val_loss: 0.5112 - val_accuracy:

In [51]:
model4.evaluate(test_x, test_y)

11/11 [==============================] - 1s 48ms/step - loss: 0.9017 - accuracy: 0.6886


[0.9016781449317932, 0.6885714530944824]

## acc 0.69
|layer|node|activation|
|-|-|-|
|conv|32|relu|
|conv|64|relu|
|lstm|32|tanh|
|lstm|32|tanh|
|dense|5|softmax|

In [55]:
model5 = Sequential()
model5.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Flatten())
model5.add(Reshape((-1, 64)))
model5.add(LSTM(32, activation="tanh", return_sequences=True))
model5.add(LSTM(32, activation="tanh"))
model5.add(Dense(5, activation="softmax"))
model5.build(input_shape=input_shape)
model5.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 50, 100, 32)       608       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 25, 50, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 12, 25, 64)       0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 19200)             0         
                                                                 
 reshape_6 (Reshape)         (None, 300, 64)          

In [56]:
model5.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [57]:
history5 = model5.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 14s 329ms/step - loss: 1.6153 - accuracy: 0.2223 - val_loss: 1.5941 - val_accuracy: 0.2214
Epoch 2/100
35/35 [==============================] - 10s 284ms/step - loss: 1.5925 - accuracy: 0.2518 - val_loss: 1.5807 - val_accuracy: 0.3107
Epoch 3/100
35/35 [==============================] - 11s 304ms/step - loss: 1.5025 - accuracy: 0.2955 - val_loss: 1.5665 - val_accuracy: 0.2929
Epoch 4/100
35/35 [==============================] - 11s 302ms/step - loss: 1.6529 - accuracy: 0.2286 - val_loss: 1.5953 - val_accuracy: 0.2393
Epoch 5/100
35/35 [==============================] - 11s 301ms/step - loss: 1.5950 - accuracy: 0.2589 - val_loss: 1.6441 - val_accuracy: 0.1786
Epoch 6/100
35/35 [==============================] - 11s 301ms/step - loss: 1.5834 - accuracy: 0.2598 - val_loss: 1.5529 - val_accuracy: 0.3429
Epoch 7/100
35/35 [==============================] - 11s 301ms/step - loss: 1.5723 - accuracy: 0.2741 - val_loss: 1.6039 - val_accuracy:

35/35 [==============================] - 11s 302ms/step - loss: 1.2434 - accuracy: 0.4812 - val_loss: 1.2529 - val_accuracy: 0.4750
Epoch 58/100
35/35 [==============================] - 11s 302ms/step - loss: 1.2529 - accuracy: 0.4893 - val_loss: 1.3206 - val_accuracy: 0.4143
Epoch 59/100
35/35 [==============================] - 11s 302ms/step - loss: 1.2606 - accuracy: 0.4679 - val_loss: 1.3049 - val_accuracy: 0.4536
Epoch 60/100
35/35 [==============================] - 11s 301ms/step - loss: 1.2078 - accuracy: 0.5000 - val_loss: 1.2325 - val_accuracy: 0.4536
Epoch 61/100
35/35 [==============================] - 11s 303ms/step - loss: 1.1460 - accuracy: 0.5268 - val_loss: 1.1675 - val_accuracy: 0.5321
Epoch 62/100
35/35 [==============================] - 11s 302ms/step - loss: 1.1332 - accuracy: 0.5634 - val_loss: 1.3030 - val_accuracy: 0.4714
Epoch 63/100
35/35 [==============================] - 11s 303ms/step - loss: 1.1328 - accuracy: 0.5304 - val_loss: 1.2432 - val_accuracy: 0.464

In [58]:
model5.evaluate(test_x, test_y)

11/11 [==============================] - 1s 92ms/step - loss: 0.8794 - accuracy: 0.6886


[0.8794440627098083, 0.6885714530944824]